# Import

In [8]:
# imoprt GYM stuff
import gym
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete

# import helpers
import numpy as np
import random
import os

# import stables_baselines stuff
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# Types of spaces

In [8]:
Discrete(3).sample()  # gives value {0,1,2}

0

In [18]:
Box(0, 1, shape=(3, 3)).sample()  # gives [3x3] with values in [0,1]

array([[0.11273727, 0.41468233, 0.31470817],
       [0.40969324, 0.33796343, 0.8231319 ],
       [0.43439248, 0.3359349 , 0.5205015 ]], dtype=float32)

In [23]:
Tuple((Discrete(3), Box(0, 1, shape=(3,)))).sample()

(2, array([0.9480498 , 0.69824386, 0.72023374], dtype=float32))

In [82]:
Dict({
    'height': Discrete(2),
    'speed': Box(0, 100, shape=(1,)),
    'color': Box(0, 255, shape=(4,))
}).sample()

OrderedDict([('color',
              array([228.14218,  91.08599,  95.28736,  33.99515], dtype=float32)),
             ('height', 1),
             ('speed', array([81.23056], dtype=float32))])

In [38]:
MultiBinary(4).sample()

array([1, 1, 0, 1], dtype=int8)

In [78]:
MultiDiscrete([4, 2]).sample()

array([3, 0])

# Building an environment

- build an agent to give us the best shower possible
- random temperature 
    - reward if $\in [37, 39]$ degrees

In [31]:
class ShowerEnv(Env):
    def __init__(self) -> None:
        self.action_space = Discrete(3)  # temp down/unchanged/up
        self.observation_space = Box(low=0, high=100, shape=(1,))
        self.state = 38 + random.randint(-3, 3)
        self.shower_length = 60  # seconds


    def step(self, action: int) -> tuple[float, float, bool, dict]:
        self.state += action-1   # apply temp adj
        self.shower_length -= 1  # decrease shower time
        reward = 1 if int(self.state) in range(37, 39) else -1
        done = self.shower_length <= 0
        info = {}

        return self.state, reward, done, info


    def reset(self) -> float:
        self.state = 38 + random.randint(-3, 3)
        self.shower_length = 60
        return self.state


    def render(self): pass

In [32]:
env = ShowerEnv()

# Test Environment

In [62]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
    print(f"Episode: {episode}, Score: {score}")
env.close()

Episode: 1, Score: -48
Episode: 2, Score: -32
Episode: 3, Score: -16
Episode: 4, Score: -38
Episode: 5, Score: -60


# Train Model

In [63]:
log_path = os.path.join('../output', 'training', 'logs')
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [65]:
model.learn(total_timesteps=100_000)

Logging to ../output/training/logs/PPO_8
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -37.7    |
| time/              |          |
|    fps             | 900      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2048     |
---------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 60         |
|    ep_rew_mean          | -36.8      |
| time/                   |            |
|    fps                  | 611        |
|    iterations           | 2          |
|    time_elapsed         | 6          |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.00975883 |
|    clip_fraction        | 0.0254     |
|    clip_range           | 0.2        |
|    entropy_loss         | -1.09      |
|    explained_variance   | 0.000226   |
|   

# Save & evaluate model

In [66]:
save_path = os.path.join('../output', 'training', 'saved models', 'ppo_shower')
model.save(save_path)

In [67]:
del model
model = PPO.load(save_path)

In [86]:
evaluate_policy(model, env, n_eval_episodes=100)

(-22.8, 55.499189183266466)